In [3]:
from sympy import symbols
from typing import Dict

class RecurrenceSubstitutionSolver:
    """
    Pure substitution method - unroll recurrence relations until reaching base case
    """
    
    def __init__(self):
        self.n = symbols('n', positive=True, integer=True)
        self.k = symbols('k', positive=True, integer=True)
        self.i = symbols('i', integer=True)
        self.steps = []
        
    def clear_steps(self):
        """Clear the step history"""
        self.steps = []
    
    def add_step(self, level: int, description: str, expression: str, explanation: str = ""):
        """Add a substitution step"""
        step = {
            'level': level,
            'description': description,
            'expression': expression,
            'explanation': explanation
        }
        self.steps.append(step)
        
    def print_step(self, step: Dict):
        """Print a single step with formatting"""
        indent = "  " * step['level']
        print(f"\n{indent}Level {step['level']}: {step['description']}")
        print(f"{indent}  {step['expression']}")
        if step['explanation']:
            print(f"{indent}  → {step['explanation']}")
    
    def print_all_steps(self):
        """Print all recorded steps"""
        print("\n" + "="*70)
        print("SUBSTITUTION METHOD - UNROLLING TO BASE CASE")
        print("="*70)
        
        for step in self.steps:
            self.print_step(step)
        
        print("\n" + "="*70)
    
    def substitute_recurrence(self, a: int, b: int, f_expr, base_case_value=1, max_levels=None):
        """
        Substitute recurrence T(n) = a*T(n/b) + f(n) until reaching T(1)
        
        Args:
            a: coefficient of recursive term
            b: divisor in recursive call  
            f_expr: the non-recursive part f(n)
            base_case_value: value of T(1)
            max_levels: maximum substitution levels (auto-detect if None)
        """
        self.clear_steps()
        
        # Determine number of levels needed to reach base case
        if max_levels is None:
            # We need n/b^k = 1, so k = log_b(n)
            max_levels = 5  # Show first 5 levels explicitly
        
        # Step 0: Original recurrence
        original = f"T(n) = {a}*T(n/{b}) + {self._format_function(f_expr)}"
        self.add_step(0, "Original Recurrence", original, 
                     "Starting with the given recurrence relation")
        
        # Step 1: First substitution
        level1_substitution = f"T(n/{b}) = {a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}"
        level1_result = f"T(n) = {a}*[{a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}] + {self._format_function(f_expr)}"
        level1_expanded = f"T(n) = {a**2}*T(n/{b**2}) + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(1, "First Substitution", level1_substitution,
                     f"Substitute T(n/{b}) using the recurrence relation")
        self.add_step(1, "Substitute Back", level1_result,
                     "Replace T(n/b) in the original equation")  
        self.add_step(1, "Expand", level1_expanded,
                     "Distribute and simplify")
        
        # Step 2: Second substitution
        level2_substitution = f"T(n/{b**2}) = {a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}"
        level2_result = f"T(n) = {a**2}*[{a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}] + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        level2_expanded = f"T(n) = {a**3}*T(n/{b**3}) + {a**2}*{self._format_function(f_expr, f'n/{b**2}')} + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(2, "Second Substitution", level2_substitution,
                     f"Continue substituting T(n/{b**2})")
        self.add_step(2, "Substitute Back", level2_result,
                     "Replace in the expanded expression")
        self.add_step(2, "Expand", level2_expanded,
                     "Distribute and collect terms")
        
        # Step 3: Pattern recognition
        pattern = f"T(n) = {a}^k * T(n/{b}^k) + Σ(i=0 to k-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        self.add_step(3, "Pattern Recognition", pattern,
                     "After k substitutions, we get this general form")
        
        # Step 4: Determine when to stop (base case)
        base_condition = f"n/{b}^k = 1"
        solve_k = f"k = log_{b}(n)"
        self.add_step(4, "Base Case Condition", base_condition,
                     "We stop when the argument reaches 1")
        self.add_step(4, "Solve for k", solve_k,
                     "Number of substitutions needed")
        
        # Step 5: Substitute base case
        base_substitution = f"T(n) = {a}^(log_{b}(n)) * T(1) + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        with_base_value = f"T(n) = {a}^(log_{b}(n)) * {base_case_value} + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(5, "Apply Base Case", base_substitution,
                     "Replace T(1) with its value")
        self.add_step(5, "With Base Value", with_base_value,
                     f"T(1) = {base_case_value}")
        
        # Step 6: Simplify the first term
        if a == b:
            first_term_simplified = f"{a}^(log_{b}(n)) = n"
        else:
            first_term_simplified = f"{a}^(log_{b}(n)) = n^(log_{b}({a}))"
        
        self.add_step(6, "Simplify First Term", first_term_simplified,
                     "Use the property a^(log_b(n)) = n^(log_b(a))")
        
        # Step 7: Evaluate the sum based on f(n)
        self._evaluate_sum(f_expr, a, b, base_case_value)
        
        return self.steps
    
    def _format_function(self, f_expr, arg='n'):
        """Format the function expression with given argument"""
        if f_expr == 1:
            return "1"
        elif f_expr == self.n:
            return arg
        elif f_expr == self.n**2:
            return f"({arg})²"
        elif f_expr == self.n**3:
            return f"({arg})³"
        elif str(f_expr) == "n*log(n)":
            return f"{arg}*log({arg})"
        else:
            # Replace n with the argument
            return str(f_expr).replace('n', arg)
    
    def _evaluate_sum(self, f_expr, a, b, base_case_value):
        """Evaluate the sum part based on the type of f(n)"""
        
        if f_expr == 1:  # f(n) = 1 (constant)
            self._evaluate_constant_sum(a, b, base_case_value)
        elif f_expr == self.n:  # f(n) = n (linear)
            self._evaluate_linear_sum(a, b, base_case_value)
        elif f_expr == self.n**2:  # f(n) = n² (quadratic)
            self._evaluate_quadratic_sum(a, b, base_case_value)
        else:
            self._evaluate_general_sum(f_expr, a, b, base_case_value)
    
    def _evaluate_constant_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = 1"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i"
        self.add_step(7, "Sum with f(n) = 1", sum_expr,
                     "The sum becomes a geometric series")
        
        if a == 1:
            result = f"log_{b}(n)"
            complexity = "O(log n)"
        else:
            geometric_formula = f"({a}^(log_{b}(n)) - 1) / ({a} - 1)"
            if a == b:
                result = f"(n - 1) / ({a} - 1)"
                complexity = "O(n)"
            else:
                result = f"(n^(log_{b}({a})) - 1) / ({a} - 1)"
                if a > b:
                    complexity = f"O(n^(log_{b}({a})))"
                else:
                    complexity = "O(1)"
        
        self.add_step(7, "Geometric Series Result", result,
                     "Using geometric series formula")
        
        # Final result
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + (n-1)/({a}-1) = O(n)"
            else:
                final = f"T(n) = {base_case_value}*n + (n-1)/({a}-1) = O(n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_linear_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)"
        factored = f"n * Σ(i=0 to log_{b}(n)-1) ({a}/{b})^i"
        
        self.add_step(7, "Sum with f(n) = n", sum_expr,
                     "Extract n and evaluate the ratio sum")
        self.add_step(7, "Factor out n", factored,
                     "Factor out n from the sum")
        
        ratio = a / b
        if ratio == 1:  # a = b
            result = f"n * log_{b}(n)"
            complexity = "O(n log n)"
        elif ratio > 1:  # a > b
            geometric_result = f"n * ({ratio}^(log_{b}(n)) - 1) / ({ratio} - 1)"
            # Since ratio^(log_b(n)) = (a/b)^(log_b(n)) = n^(log_b(a/b)) = n^(log_b(a) - 1)
            simplified = f"n * (n^(log_{b}({a}) - 1) - 1) / ({ratio} - 1)"
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b
            result = f"n * (1 - ({ratio})^(log_{b}(n))) / (1 - {ratio})"
            complexity = "O(n)"
        
        self.add_step(7, "Evaluate Geometric Series", result,
                     f"Ratio r = {a}/{b} = {ratio}")
        
        # Final result combining both terms
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + n*log_{b}(n) = O(n log n)"
            else:
                final = f"T(n) = {base_case_value}*n + n*log_{b}(n) = O(n log n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_quadratic_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n²"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)²"
        factored = f"n² * Σ(i=0 to log_{b}(n)-1) ({a}/{b}²)^i"
        
        self.add_step(7, "Sum with f(n) = n²", sum_expr,
                     "Extract n² and evaluate the ratio sum")
        self.add_step(7, "Factor out n²", factored,
                     "Factor out n² from the sum")
        
        ratio = a / (b**2)
        if abs(ratio - 1) < 1e-10:  # a = b²
            result = f"n² * log_{b}(n)"
            complexity = "O(n² log n)"
        elif ratio > 1:  # a > b²
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b²
            complexity = "O(n²)"
        
        self.add_step(7, "Evaluate Sum", f"Ratio = {a}/{b}² = {ratio}",
                     "Determine the dominant term")
        
        final = f"T(n) = {complexity}"
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_general_sum(self, f_expr, a, b, base_case_value):
        """Handle general f(n) expressions"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(7, f"Sum with f(n) = {f_expr}", sum_expr,
                     "Evaluate this sum based on the specific function")
        
        final = "T(n) = [Depends on specific analysis of the sum]"
        self.add_step(8, "Final Result", final, 
                     "Complete analysis requires evaluating the specific sum")

def demonstrate_substitution_examples():
    """
    Demonstrate pure substitution method on classic examples
    """
    solver = RecurrenceSubstitutionSolver()
    
    print("PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE")
    print("=" * 60)
    
    # Example 1: Binary Search - T(n) = T(n/2) + 1
    print("\nEXAMPLE 1: BINARY SEARCH")
    print("T(n) = T(n/2) + 1, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=1, b=2, f_expr=1, base_case_value=1)
    solver.print_all_steps()
    
    # Example 2: Merge Sort - T(n) = 2T(n/2) + n  
    print("\n\nEXAMPLE 2: MERGE SORT")
    print("T(n) = 2T(n/2) + n, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=2, b=2, f_expr=solver.n, base_case_value=1)
    solver.print_all_steps()


if __name__ == "__main__":
    demonstrate_substitution_examples()

PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE

EXAMPLE 1: BINARY SEARCH
T(n) = T(n/2) + 1, T(1) = 1
----------------------------------------

SUBSTITUTION METHOD - UNROLLING TO BASE CASE

Level 0: Original Recurrence
  T(n) = 1*T(n/2) + 1
  → Starting with the given recurrence relation

  Level 1: First Substitution
    T(n/2) = 1*T(n/4) + 1
    → Substitute T(n/2) using the recurrence relation

  Level 1: Substitute Back
    T(n) = 1*[1*T(n/4) + 1] + 1
    → Replace T(n/b) in the original equation

  Level 1: Expand
    T(n) = 1*T(n/4) + 1*1 + 1
    → Distribute and simplify

    Level 2: Second Substitution
      T(n/4) = 1*T(n/8) + 1
      → Continue substituting T(n/4)

    Level 2: Substitute Back
      T(n) = 1*[1*T(n/8) + 1] + 1*1 + 1
      → Replace in the expanded expression

    Level 2: Expand
      T(n) = 1*T(n/8) + 1*1 + 1*1 + 1
      → Distribute and collect terms

      Level 3: Pattern Recognition
        T(n) = 1^k * T(n/2^k) + Σ(i=0 to k-1) 1^i * 1
        → After k 

In [6]:
import sympy as sp
from sympy import symbols, log, simplify, expand, factor, ceiling, floor, Sum, Pow
from typing import Dict, List, Tuple
import math

class RecurrenceSubstitutionSolver:
    """
    Pure substitution method - unroll recurrence relations until reaching base case
    """
    
    def __init__(self):
        self.n = symbols('n', positive=True, integer=True)
        self.k = symbols('k', positive=True, integer=True)
        self.i = symbols('i', integer=True)
        self.steps = []
        
    def clear_steps(self):
        """Clear the step history"""
        self.steps = []
    
    def add_step(self, level: int, description: str, expression: str, explanation: str = ""):
        """Add a substitution step"""
        step = {
            'level': level,
            'description': description,
            'expression': expression,
            'explanation': explanation
        }
        self.steps.append(step)
        
    def print_step(self, step: Dict):
        """Print a single step with formatting"""
        indent = "  " * step['level']
        print(f"\n{indent}Level {step['level']}: {step['description']}")
        print(f"{indent}  {step['expression']}")
        if step['explanation']:
            print(f"{indent}  → {step['explanation']}")
    
    def print_all_steps(self):
        """Print all recorded steps"""
        print("\n" + "="*70)
        print("SUBSTITUTION METHOD - UNROLLING TO BASE CASE")
        print("="*70)
        
        for step in self.steps:
            self.print_step(step)
        
        print("\n" + "="*70)
    
    def substitute_recurrence(self, a: int, b: int, f_expr, base_case_value=1, max_levels=None):
        """
        Substitute recurrence T(n) = a*T(n/b) + f(n) until reaching T(1)
        
        Args:
            a: coefficient of recursive term
            b: divisor in recursive call  
            f_expr: the non-recursive part f(n)
            base_case_value: value of T(1)
            max_levels: maximum substitution levels (auto-detect if None)
        """
        self.clear_steps()
        
        # Determine number of levels needed to reach base case
        if max_levels is None:
            # We need n/b^k = 1, so k = log_b(n)
            max_levels = 5  # Show first 5 levels explicitly
        
        # Step 0: Original recurrence
        original = f"T(n) = {a}*T(n/{b}) + {self._format_function(f_expr)}"
        self.add_step(0, "Original Recurrence", original, 
                     "Starting with the given recurrence relation")
        
        # Step 1: First substitution
        level1_substitution = f"T(n/{b}) = {a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}"
        level1_result = f"T(n) = {a}*[{a}*T(n/{b**2}) + {self._format_function(f_expr, f'n/{b}')}] + {self._format_function(f_expr)}"
        level1_expanded = f"T(n) = {a**2}*T(n/{b**2}) + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(1, "First Substitution", level1_substitution,
                     f"Substitute T(n/{b}) using the recurrence relation")
        self.add_step(1, "Substitute Back", level1_result,
                     "Replace T(n/b) in the original equation")  
        self.add_step(1, "Expand", level1_expanded,
                     "Distribute and simplify")
        
        # Step 2: Second substitution
        level2_substitution = f"T(n/{b**2}) = {a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}"
        level2_result = f"T(n) = {a**2}*[{a}*T(n/{b**3}) + {self._format_function(f_expr, f'n/{b**2}')}] + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        level2_expanded = f"T(n) = {a**3}*T(n/{b**3}) + {a**2}*{self._format_function(f_expr, f'n/{b**2}')} + {a}*{self._format_function(f_expr, f'n/{b}')} + {self._format_function(f_expr)}"
        
        self.add_step(2, "Second Substitution", level2_substitution,
                     f"Continue substituting T(n/{b**2})")
        self.add_step(2, "Substitute Back", level2_result,
                     "Replace in the expanded expression")
        self.add_step(2, "Expand", level2_expanded,
                     "Distribute and collect terms")
        
        # Step 3: Pattern recognition
        pattern = f"T(n) = {a}^k * T(n/{b}^k) + Σ(i=0 to k-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        self.add_step(3, "Pattern Recognition", pattern,
                     "After k substitutions, we get this general form")
        
        # Step 4: Determine when to stop (base case)
        base_condition = f"n/{b}^k = 1"
        solve_k = f"k = log_{b}(n)"
        self.add_step(4, "Base Case Condition", base_condition,
                     f"We stop when the argument reaches 1")
        self.add_step(4, "Solve for k", solve_k,
                     f"Number of substitutions needed")
        
        # Step 5: Substitute base case
        base_substitution = f"T(n) = {a}^(log_{b}(n)) * T(1) + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        with_base_value = f"T(n) = {a}^(log_{b}(n)) * {base_case_value} + Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(5, "Apply Base Case", base_substitution,
                     f"Replace T(1) with its value")
        self.add_step(5, "With Base Value", with_base_value,
                     f"T(1) = {base_case_value}")
        
        # Step 6: Simplify the first term
        if a == 1:
            first_term_simplified = f"{a}^(log_{b}(n)) = 1"
            explanation = f"Since 1 raised to any power equals 1"
        elif a == b:
            first_term_simplified = f"{a}^(log_{b}(n)) = n"
            explanation = f"Since {a}^(log_{a}(n)) = n"
        else:
            first_term_simplified = f"{a}^(log_{b}(n)) = n^(log_{b}({a}))"
            explanation = f"Use the property a^(log_b(n)) = n^(log_b(a))"
        
        self.add_step(6, "Simplify First Term", first_term_simplified, explanation)
        
        # Step 7: Evaluate the sum based on f(n)
        self._evaluate_sum(f_expr, a, b, base_case_value)
        
        return self.steps
    
    def _format_function(self, f_expr, arg='n'):
        """Format the function expression with given argument"""
        if f_expr == 1:
            return "1"
        elif f_expr == self.n:
            return arg
        elif f_expr == self.n**2:
            return f"({arg})²"
        elif f_expr == self.n**3:
            return f"({arg})³"
        elif str(f_expr) == "n*log(n)":
            return f"{arg}*log({arg})"
        else:
            # Replace n with the argument
            return str(f_expr).replace('n', arg)
    
    def _evaluate_sum(self, f_expr, a, b, base_case_value):
        """Evaluate the sum part based on the type of f(n)"""
        
        if f_expr == 1:  # f(n) = 1 (constant)
            self._evaluate_constant_sum(a, b, base_case_value)
        elif f_expr == self.n:  # f(n) = n (linear)
            self._evaluate_linear_sum(a, b, base_case_value)
        elif f_expr == self.n**2:  # f(n) = n² (quadratic)
            self._evaluate_quadratic_sum(a, b, base_case_value)
        else:
            self._evaluate_general_sum(f_expr, a, b, base_case_value)
    
    def _evaluate_constant_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = 1"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i"
        self.add_step(7, "Sum with f(n) = 1", sum_expr,
                     "The sum becomes a geometric series")
        
        if a == 1:
            result = f"log_{b}(n)"
            complexity = "O(log n)"
            geometric_explanation = f"Since {a}^i = 1 for all i, we sum 1 exactly log_{b}(n) times"
        else:
            geometric_formula = f"({a}^(log_{b}(n)) - 1) / ({a} - 1)"
            if a == b:
                result = f"(n - 1) / ({a} - 1)"
                complexity = "O(n)"
                geometric_explanation = f"Geometric series with ratio {a}, sum = {geometric_formula}"
            else:
                result = f"(n^(log_{b}({a})) - 1) / ({a} - 1)"
                if a > b:
                    complexity = f"O(n^(log_{b}({a})))"
                else:
                    complexity = "O(1)"
                geometric_explanation = f"Geometric series with ratio {a}, sum = {geometric_formula}"
        
        self.add_step(7, "Geometric Series Result", result, geometric_explanation)
        
        # Final result
        if a == 1:
            if base_case_value == 1:
                final = f"T(n) = 1 + log_{b}(n) = O(log n)"
            else:
                final = f"T(n) = {base_case_value} + log_{b}(n) = O(log n)"
        elif a == b:
            if base_case_value == 1:
                final = f"T(n) = n + (n-1)/({a}-1) = O(n)"
            else:
                final = f"T(n) = {base_case_value}*n + (n-1)/({a}-1) = O(n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_linear_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)"
        factored = f"n * Σ(i=0 to log_{b}(n)-1) ({a}/{b})^i"
        
        self.add_step(7, "Sum with f(n) = n", sum_expr,
                     "Extract n and evaluate the ratio sum")
        self.add_step(7, "Factor out n", factored,
                     f"Factor out n from the sum")
        
        ratio = a / b
        if ratio == 1:  # a = b
            result = f"n * log_{b}(n)"
            complexity = "O(n log n)"
        elif ratio > 1:  # a > b
            geometric_result = f"n * ({ratio}^(log_{b}(n)) - 1) / ({ratio} - 1)"
            # Since ratio^(log_b(n)) = (a/b)^(log_b(n)) = n^(log_b(a/b)) = n^(log_b(a) - 1)
            simplified = f"n * (n^(log_{b}({a}) - 1) - 1) / ({ratio} - 1)"
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b
            result = f"n * (1 - ({ratio})^(log_{b}(n))) / (1 - {ratio})"
            complexity = "O(n)"
        
        self.add_step(7, "Evaluate Geometric Series", result,
                     f"Ratio r = {a}/{b} = {ratio}")
        
        # Final result combining both terms
        if a == b:
            if base_case_value == 1:
                final = f"T(n) = n + n*log_{b}(n) = O(n log n)"
            else:
                final = f"T(n) = {base_case_value}*n + n*log_{b}(n) = O(n log n)"
        else:
            final = f"T(n) = {complexity}"
        
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_quadratic_sum(self, a, b, base_case_value):
        """Evaluate sum when f(n) = n²"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * (n/{b}^i)²"
        factored = f"n² * Σ(i=0 to log_{b}(n)-1) ({a}/{b}²)^i"
        
        self.add_step(7, "Sum with f(n) = n²", sum_expr,
                     "Extract n² and evaluate the ratio sum")
        self.add_step(7, "Factor out n²", factored,
                     f"Factor out n² from the sum")
        
        ratio = a / (b**2)
        if abs(ratio - 1) < 1e-10:  # a = b²
            result = f"n² * log_{b}(n)"
            complexity = "O(n² log n)"
        elif ratio > 1:  # a > b²
            complexity = f"O(n^(log_{b}({a})))"
        else:  # a < b²
            complexity = "O(n²)"
        
        self.add_step(7, "Evaluate Sum", f"Ratio = {a}/{b}² = {ratio}",
                     f"Determine the dominant term")
        
        final = f"T(n) = {complexity}"
        self.add_step(8, "Final Result", final, "Complete solution")
    
    def _evaluate_general_sum(self, f_expr, a, b, base_case_value):
        """Handle general f(n) expressions"""
        
        sum_expr = f"Σ(i=0 to log_{b}(n)-1) {a}^i * {self._format_function(f_expr, f'n/{b}^i')}"
        
        self.add_step(7, f"Sum with f(n) = {f_expr}", sum_expr,
                     "Evaluate this sum based on the specific function")
        
        final = "T(n) = [Depends on specific analysis of the sum]"
        self.add_step(8, "Final Result", final, 
                     "Complete analysis requires evaluating the specific sum")

def demonstrate_substitution_examples():
    """
    Demonstrate pure substitution method on classic examples
    """
    solver = RecurrenceSubstitutionSolver()
    
    print("PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE")
    print("=" * 60)
    
    # Example 1: Binary Search - T(n) = T(n/2) + 1
    print("\nEXAMPLE 1: BINARY SEARCH")
    print("T(n) = T(n/2) + 1, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=1, b=2, f_expr=1, base_case_value=1)
    solver.print_all_steps()
    
    # Example 2: Merge Sort - T(n) = 2T(n/2) + n  
    print("\n\nEXAMPLE 2: MERGE SORT")
    print("T(n) = 2T(n/2) + n, T(1) = 1")
    print("-" * 40)
    
    solver.substitute_recurrence(a=2, b=2, f_expr=solver.n, base_case_value=1)
    solver.print_all_steps()
    

if __name__ == "__main__":
    demonstrate_substitution_examples()

PURE SUBSTITUTION METHOD - UNROLL TO BASE CASE

EXAMPLE 1: BINARY SEARCH
T(n) = T(n/2) + 1, T(1) = 1
----------------------------------------

SUBSTITUTION METHOD - UNROLLING TO BASE CASE

Level 0: Original Recurrence
  T(n) = 1*T(n/2) + 1
  → Starting with the given recurrence relation

  Level 1: First Substitution
    T(n/2) = 1*T(n/4) + 1
    → Substitute T(n/2) using the recurrence relation

  Level 1: Substitute Back
    T(n) = 1*[1*T(n/4) + 1] + 1
    → Replace T(n/b) in the original equation

  Level 1: Expand
    T(n) = 1*T(n/4) + 1*1 + 1
    → Distribute and simplify

    Level 2: Second Substitution
      T(n/4) = 1*T(n/8) + 1
      → Continue substituting T(n/4)

    Level 2: Substitute Back
      T(n) = 1*[1*T(n/8) + 1] + 1*1 + 1
      → Replace in the expanded expression

    Level 2: Expand
      T(n) = 1*T(n/8) + 1*1 + 1*1 + 1
      → Distribute and collect terms

      Level 3: Pattern Recognition
        T(n) = 1^k * T(n/2^k) + Σ(i=0 to k-1) 1^i * 1
        → After k 

In [2]:
import sympy as sp
from sympy import symbols, log, simplify, expand, factor, ceiling, floor, Sum, Pow, latex
from typing import Dict, List, Tuple, Optional, Union, Callable
import math
from fractions import Fraction
from abc import ABC, abstractmethod

class RecurrenceStep:
    """Represents a single step in the solution process"""
    
    def __init__(self, level: int, title: str, expression: str, 
                 explanation: str = "", latex_expr: str = "", 
                 step_type: str = "calculation"):
        self.level = level
        self.title = title
        self.expression = expression
        self.explanation = explanation
        self.latex_expr = latex_expr
        self.step_type = step_type  # 'calculation', 'insight', 'pattern', 'conclusion'
        
    def __str__(self):
        indent = "  " * self.level
        result = f"\n{indent}{'📍' if self.step_type == 'insight' else '🔢'} {self.title}"
        result += f"\n{indent}  {self.expression}"
        if self.explanation:
            result += f"\n{indent}  💡 {self.explanation}"
        return result

class RecurrenceRelation:
    """Represents a recurrence relation with validation"""
    
    def __init__(self, a: Union[int, float], b: Union[int, float], 
                 f_expr, base_cases: Dict[int, Union[int, float]] = None,
                 name: str = "T"):
        self.a = a  # coefficient of recursive term
        self.b = b  # division factor
        self.f_expr = f_expr  # non-recursive part
        self.base_cases = base_cases or {1: 1}
        self.name = name
        
        self._validate()
    
    def _validate(self):
        """Validate the recurrence relation parameters"""
        if self.b <= 1:
            raise ValueError("Division factor b must be > 1")
        if self.a <= 0:
            raise ValueError("Coefficient a must be positive")
        
    def __str__(self):
        base_str = ", ".join([f"{self.name}({k}) = {v}" for k, v in self.base_cases.items()])
        return f"{self.name}(n) = {self.a}·{self.name}(n/{self.b}) + {self.f_expr}, {base_str}"

class VisualizationHelper:
    """Helper class for creating visual representations"""
    
    @staticmethod
    def create_recursion_tree(levels: int, branching_factor: int, division_factor: int = 2):
        """Create a simple ASCII recursion tree"""
        tree = []
        for level in range(levels):
            indent = "  " * level
            nodes = branching_factor ** level
            if level == 0:
                tree.append(f"{indent}T(n)")
            else:
                # Create representative node sizes for this level
                if division_factor ** level == 1:
                    node_size = "n"
                else:
                    node_size = f"n/{division_factor**level}"
                
                if nodes <= 4:
                    node_list = [f"T({node_size})" for _ in range(nodes)]
                    tree.append(f"{indent}{'  '.join(node_list)}")
                else:
                    tree.append(f"{indent}T({node_size}) T({node_size}) ... T({node_size}) ({nodes} nodes)")
        
        if levels < 5:
            tree.append(f"{'  ' * levels}Base cases: T(1) = constant")
        
        return "\n".join(tree)

class RecurrenceSubstitutionSolver:
    """
    Enhanced pedagogical solver for recurrence relations using pure substitution method.
    Provides step-by-step solutions with detailed explanations and multiple output formats.
    """
    
    def __init__(self, show_patterns: bool = True, show_insights: bool = True, 
                 max_substitution_levels: int = 4):
        self.n = symbols('n', positive=True, integer=True)
        self.k = symbols('k', positive=True, integer=True)
        self.i = symbols('i', integer=True)
        self.steps: List[RecurrenceStep] = []
        self.show_patterns = show_patterns
        self.show_insights = show_insights
        self.max_substitution_levels = max_substitution_levels
        
    def solve(self, recurrence: RecurrenceRelation, show_tree: bool = True) -> List[RecurrenceStep]:
        """
        Main solving method that orchestrates the entire solution process
        """
        self.steps = []
        self.recurrence = recurrence
        
        # Phase 1: Setup and introduction
        self._introduce_problem(show_tree)
        
        # Phase 2: Perform substitutions
        self._perform_substitutions()
        
        # Phase 3: Pattern recognition
        if self.show_patterns:
            self._identify_patterns()
        
        # Phase 4: Base case analysis
        self._analyze_base_case()
        
        # Phase 5: Sum evaluation
        self._evaluate_sum()
        
        # Phase 6: Final complexity analysis
        self._analyze_complexity()
        
        # Phase 7: Insights and generalizations
        if self.show_insights:
            self._provide_insights()
        
        return self.steps
    
    def _introduce_problem(self, show_tree: bool):
        """Introduce the recurrence relation and provide context"""
        self._add_step(0, "Problem Setup", str(self.recurrence), 
                      "We'll solve this recurrence using the substitution method (unrolling)",
                      step_type="insight")
        
        if show_tree:
            tree = VisualizationHelper.create_recursion_tree(
                4, self.recurrence.a, self.recurrence.b
            )
            self._add_step(0, "Recursion Tree Structure", tree,
                          "This shows how the problem breaks down recursively",
                          step_type="insight")
    
    def _perform_substitutions(self):
        """Perform the actual substitution steps"""
        a, b = self.recurrence.a, self.recurrence.b
        f_expr = self.recurrence.f_expr
        T = self.recurrence.name
        
        # Level 0: Original
        original = f"{T}(n) = {a}·{T}(n/{b}) + {self._format_function(f_expr)}"
        self._add_step(1, "Original Recurrence", original)
        
        # Perform substitutions
        for level in range(1, min(self.max_substitution_levels + 1, 4)):
            self._perform_single_substitution(level, a, b, f_expr, T)
    
    def _perform_single_substitution(self, level: int, a: int, b: int, f_expr, T: str):
        """Perform a single substitution step with detailed explanation"""
        
        # Show what we're substituting
        sub_expr = f"{T}(n/{b**level}) = {a}·{T}(n/{b**(level+1)}) + {self._format_function(f_expr, f'n/{b**level}')}"
        self._add_step(level+1, f"Substitution {level}", sub_expr,
                      f"Apply the recurrence relation to {T}(n/{b**level})")
        
        # Show the substitution back into the main equation
        terms = []
        
        # Recursive term
        recursive_coeff = a**(level+1)
        recursive_term = f"{recursive_coeff}·{T}(n/{b**(level+1)})"
        
        # Non-recursive terms (the sum so far)
        for i in range(level+1):
            coeff = a**i
            if coeff == 1:
                term = self._format_function(f_expr, f"n/{b**i}" if i > 0 else "n")
            else:
                term = f"{coeff}·{self._format_function(f_expr, f'n/{b**i}' if i > 0 else 'n')}"
            terms.append(term)
        
        result = f"{T}(n) = {recursive_term} + " + " + ".join(terms)
        self._add_step(level+1, f"After Substitution {level}", result,
                      "Collect all terms after substitution")
    
    def _identify_patterns(self):
        """Identify and explain the emerging patterns"""
        a, b = self.recurrence.a, self.recurrence.b
        f_expr = self.recurrence.f_expr
        T = self.recurrence.name
        
        # General pattern after k substitutions
        pattern = (f"{T}(n) = {a}^k · {T}(n/{b}^k) + "
                  f"∑(i=0 to k-1) {a}^i · {self._format_function(f_expr, 'n/b^i')}")
        
        self._add_step(2, "General Pattern Recognition", pattern,
                      "After k substitutions, this is the general form we get",
                      step_type="pattern")
        
        # Explain each component
        self._add_step(2, "Pattern Components", 
                      f"• Recursive term: {a}^k · {T}(n/{b}^k) → shrinks as k increases\n"
                      f"• Sum term: accumulates work done at each level\n"
                      f"• Total levels needed: k such that n/{b}^k = 1",
                      "Understanding the structure helps us solve the recurrence",
                      step_type="insight")
    
    def _analyze_base_case(self):
        """Analyze when we reach the base case"""
        b = self.recurrence.b
        base_value = list(self.recurrence.base_cases.values())[0]
        base_key = list(self.recurrence.base_cases.keys())[0]
        
        # When do we reach base case?
        condition = f"n/{b}^k = {base_key}"
        solution = f"k = log_{b}(n/{base_key}) = log_{b}(n)" if base_key == 1 else f"k = log_{b}(n/{base_key})"
        
        self._add_step(3, "Base Case Condition", condition,
                      f"We stop substituting when we reach {self.recurrence.name}({base_key})")
        
        self._add_step(3, "Number of Substitutions", solution,
                      "This tells us how many levels the recursion tree has")
        
        # Substitute the base case
        a = self.recurrence.a
        if base_key == 1:
            substituted = (f"{self.recurrence.name}(n) = {a}^(log_{b}(n)) · {base_value} + "
                          f"∑(i=0 to log_{b}(n)-1) {a}^i · {self._format_function(self.recurrence.f_expr, 'n/b^i')}")
        else:
            substituted = (f"{self.recurrence.name}(n) = {a}^(log_{b}(n/{base_key})) · {base_value} + "
                          f"∑(i=0 to log_{b}(n/{base_key})-1) {a}^i · {self._format_function(self.recurrence.f_expr, 'n/b^i')}")
        
        self._add_step(3, "Apply Base Case", substituted,
                      f"Replace {self.recurrence.name}({base_key}) with {base_value}")
    
    def _evaluate_sum(self):
        """Evaluate the sum based on the type of f(n)"""
        a, b = self.recurrence.a, self.recurrence.b
        f_expr = self.recurrence.f_expr
        
        # Simplify the first term
        self._simplify_first_term(a, b)
        
        # Evaluate sum based on f(n) type
        if f_expr == 1:
            self._evaluate_constant_sum(a, b)
        elif f_expr == self.n:
            self._evaluate_linear_sum(a, b)
        elif f_expr == self.n**2:
            self._evaluate_quadratic_sum(a, b)
        elif str(f_expr).startswith("n*log"):
            self._evaluate_nlogn_sum(a, b)
        else:
            self._evaluate_general_sum(f_expr, a, b)
    
    def _simplify_first_term(self, a: int, b: int):
        """Simplify the first term a^(log_b(n))"""
        if a == 1:
            simplified = "1^(log_b(n)) = 1"
            explanation = "Any number to any power equals 1 when the base is 1"
        elif a == b:
            simplified = f"{a}^(log_{a}(n)) = n"
            explanation = f"By definition of logarithm: a^(log_a(x)) = x"
        else:
            simplified = f"{a}^(log_{b}(n)) = n^(log_{b}({a}))"
            explanation = f"Using the change of base property: a^(log_b(n)) = n^(log_b(a))"
            
            # Additional insight about the exponent
            log_ratio = math.log(a) / math.log(b)
            self._add_step(4, "Exponent Value", f"log_{b}({a}) ≈ {log_ratio:.3f}",
                          f"This determines whether the first term dominates",
                          step_type="insight")
        
        self._add_step(4, "Simplify First Term", simplified, explanation)
    
    def _evaluate_constant_sum(self, a: int, b: int):
        """Handle f(n) = constant case"""
        self._add_step(4, "Sum Type: Constant", 
                      "∑(i=0 to log_b(n)-1) a^i = geometric series",
                      "When f(n) = constant, we get a geometric series")
        
        if a == 1:
            result = f"Sum = log_{b}(n) terms of 1 = log_{b}(n)"
            dominant = "O(log n)"
        else:
            result = f"Sum = (a^(log_b(n)) - 1)/(a-1) = (n^(log_b(a)) - 1)/(a-1)"
            if a == b:
                dominant = "O(n)"
            elif a > b:
                dominant = f"O(n^(log_{b}({a})))"
            else:
                dominant = "O(1)"
        
        self._add_step(4, "Geometric Series Result", result)
        self._add_step(4, "Sum Complexity", dominant, 
                      "This determines the overall complexity")
    
    def _evaluate_linear_sum(self, a: int, b: int):
        """Handle f(n) = n case"""
        self._add_step(4, "Sum Type: Linear", 
                      "∑(i=0 to log_b(n)-1) a^i · (n/b^i) = n · ∑(i=0 to log_b(n)-1) (a/b)^i",
                      "Factor out n and analyze the ratio a/b")
        
        ratio = a / b
        self._add_step(4, "Critical Ratio", f"a/b = {a}/{b} = {ratio}",
                      "This ratio determines which term dominates")
        
        if abs(ratio - 1) < 1e-10:  # a = b
            result = "Sum = n · log_b(n)"
            dominant = "O(n log n)"
        elif ratio > 1:  # a > b  
            dominant = f"O(n^(log_{b}({a})))"
            result = f"Sum dominated by geometric growth"
        else:  # a < b
            dominant = "O(n)"
            result = f"Sum = O(n) since geometric series converges"
        
        self._add_step(4, "Linear Sum Result", result)
        self._add_step(4, "Sum Complexity", dominant)
    
    def _evaluate_quadratic_sum(self, a: int, b: int):
        """Handle f(n) = n² case"""
        self._add_step(4, "Sum Type: Quadratic", 
                      "∑(i=0 to log_b(n)-1) a^i · (n/b^i)² = n² · ∑(i=0 to log_b(n)-1) (a/b²)^i",
                      "Factor out n² and analyze the ratio a/b²")
        
        ratio = a / (b**2)
        self._add_step(4, "Critical Ratio", f"a/b² = {a}/{b}² = {ratio}",
                      "Compare a with b² to determine dominance")
        
        if abs(ratio - 1) < 1e-10:  # a = b²
            dominant = "O(n² log n)"
        elif ratio > 1:  # a > b²
            dominant = f"O(n^(log_{b}({a})))"
        else:  # a < b²
            dominant = "O(n²)"
        
        self._add_step(4, "Quadratic Sum Complexity", dominant)
    
    def _evaluate_nlogn_sum(self, a: int, b: int):
        """Handle f(n) = n log n case"""
        self._add_step(4, "Sum Type: n log n", 
                      "This case requires careful analysis of logarithmic factors",
                      "The logarithmic factor adds complexity to the analysis")
        
        if a == b:
            dominant = "O(n (log n)²)"
        elif a > b:
            dominant = f"O(n^(log_{b}({a})) log n)"
        else:
            dominant = "O(n log n)"
        
        self._add_step(4, "n log n Sum Complexity", dominant)
    
    def _evaluate_general_sum(self, f_expr, a: int, b: int):
        """Handle general f(n) expressions"""
        self._add_step(4, "Sum Type: General", 
                      f"f(n) = {f_expr} requires specific analysis",
                      "Complex functions need individual treatment")
    
    def _analyze_complexity(self):
        """Provide the final complexity analysis"""
        a, b = self.recurrence.a, self.recurrence.b
        f_expr = self.recurrence.f_expr
        
        # Master theorem comparison
        log_b_a = math.log(a) / math.log(b)
        
        self._add_step(5, "Master Theorem Perspective", 
                      f"log_{b}(a) = {log_b_a:.3f}",
                      "This helps classify the recurrence type",
                      step_type="insight")
        
        # Determine which case applies
        if f_expr == 1:  # f(n) = O(1)
            if a == 1:
                complexity = "Θ(log n)"
                case = "Case 2 (a = 1, f(n) = 1)"
            elif a > 1:
                complexity = f"Θ(n^(log_{b}({a}))) = Θ(n^{log_b_a:.3f})"
                case = "Case 1 (a > 1, f(n) = O(1))"
        elif f_expr == self.n:  # f(n) = Θ(n)
            if abs(log_b_a - 1) < 1e-10:  # a = b
                complexity = "Θ(n log n)"
                case = "Case 2 (a = b, f(n) = n)"
            elif log_b_a > 1:  # a > b
                complexity = f"Θ(n^{log_b_a:.3f})"
                case = "Case 1 (a > b, f(n) = n)"
            else:  # a < b
                complexity = "Θ(n)"
                case = "Case 3 (a < b, f(n) = n)"
        else:
            complexity = "[Requires specific analysis]"
            case = "General case"
        
        self._add_step(5, "Final Complexity", complexity,
                      f"Master Theorem {case}")
    
    def _provide_insights(self):
        """Provide educational insights and generalizations"""
        insights = [
            "🎯 Key Insight: The relationship between 'a' and 'b' determines which term dominates",
            "📊 When a > b^k (where f(n) = O(n^k)), the recursive term dominates",
            "⚖️ When a = b^k, both terms contribute equally, adding a log factor", 
            "📈 When a < b^k, the sum term (work per level) dominates",
            "🌳 The recursion tree has log_b(n) levels with a^i nodes at level i"
        ]
        
        for i, insight in enumerate(insights):
            self._add_step(6, f"Educational Insight {i+1}", insight, 
                          step_type="insight")
    
    def _add_step(self, level: int, title: str, expression: str, 
                  explanation: str = "", step_type: str = "calculation"):
        """Add a step to the solution"""
        step = RecurrenceStep(level, title, expression, explanation, 
                            step_type=step_type)
        self.steps.append(step)
    
    def _format_function(self, f_expr, arg: str = 'n') -> str:
        """Format function expressions with proper mathematical notation"""
        if f_expr == 1:
            return "1"
        elif f_expr == self.n:
            return arg
        elif f_expr == self.n**2:
            return f"({arg})²"
        elif f_expr == self.n**3:
            return f"({arg})³"
        elif str(f_expr) == "n*log(n)":
            return f"{arg}·log({arg})"
        else:
            return str(f_expr).replace('n', arg)
    
    def print_solution(self, show_tree: bool = False):
        """Print the complete solution with formatting"""
        print("🔍 RECURRENCE RELATION SOLVER - SUBSTITUTION METHOD")
        print("=" * 70)
        
        for step in self.steps:
            print(step)
        
        print("\n" + "=" * 70)

def create_example_problems():
    """Create a collection of educational examples"""
    n = symbols('n', positive=True, integer=True)
    
    examples = {
        "Binary Search": RecurrenceRelation(1, 2, 1, {1: 1}, "T"),
        "Merge Sort": RecurrenceRelation(2, 2, n, {1: 1}, "T"), 
        "Master Theorem Case 1": RecurrenceRelation(4, 2, 1, {1: 1}, "T"),
        "Master Theorem Case 3": RecurrenceRelation(2, 4, n**2, {1: 1}, "T"),
        "Karatsuba Multiplication": RecurrenceRelation(3, 2, n, {1: 1}, "T"),
        "Strassen's Algorithm": RecurrenceRelation(7, 2, n**2, {1: 1}, "T"),
        "Quicksort Average": RecurrenceRelation(2, 2, n, {1: 1}, "T"),
    }
    
    return examples

def demonstrate_enhanced_solver():
    """Demonstrate the enhanced solver with multiple examples"""
    solver = RecurrenceSubstitutionSolver(show_patterns=True, show_insights=True)
    examples = create_example_problems()
    
    for name, recurrence in list(examples.items())[:3]:  # Show first 3 examples
        print(f"\n{'='*20} {name.upper()} {'='*20}")
        solver.solve(recurrence, show_tree=True)
        solver.print_solution()
        print("\n" + "🎓 " + "="*68 + "\n")

if __name__ == "__main__":
    demonstrate_enhanced_solver()


==================== BINARY SEARCH ====================
🔍 RECURRENCE RELATION SOLVER - SUBSTITUTION METHOD

📍 Problem Setup
  T(n) = 1·T(n/2) + 1, T(1) = 1
  💡 We'll solve this recurrence using the substitution method (unrolling)

📍 Recursion Tree Structure
  T(n)
  T(n/2)
    T(n/4)
      T(n/8)
        Base cases: T(1) = constant
  💡 This shows how the problem breaks down recursively

  🔢 Original Recurrence
    T(n) = 1·T(n/2) + 1

    🔢 Substitution 1
      T(n/2) = 1·T(n/4) + 1
      💡 Apply the recurrence relation to T(n/2)

    🔢 After Substitution 1
      T(n) = 1·T(n/4) + 1 + 1
      💡 Collect all terms after substitution

      🔢 Substitution 2
        T(n/4) = 1·T(n/8) + 1
        💡 Apply the recurrence relation to T(n/4)

      🔢 After Substitution 2
        T(n) = 1·T(n/8) + 1 + 1 + 1
        💡 Collect all terms after substitution

        🔢 Substitution 3
          T(n/8) = 1·T(n/16) + 1
          💡 Apply the recurrence relation to T(n/8)

        🔢 After Substitution 3
